# TBA 3102 - Text Analytics
## Practical Lab 09 - Text Summarization and Topic Models (II)
### Question 1 - Topic Modeling
Student: Nicky Ng <br>
GitHub User: [ahjimomo](https://github.com/ahjimomo) <br>
Student Number: A0194330L

### Libraries

In [1]:
# Data Wrangling
import numpy as np
import pandas as pd

# Topic Modeling
import nltk
import gensim

# Tokenizer & Feature Engineering
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')
stop_words = nltk.corpus.stopwords.words('english')
wnl = nltk.stem.wordnet.WordNetLemmatizer()

# Gemsim models
from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
from gensim.models.lsimodel import LsiModel

# Parameters Tuning
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, ParameterGrid

# Display DF
from IPython.core.display import HTML
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('max_colwidth', None)
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Import dataset

In [2]:
# Import cleaned dataset
raw_df = pd.read_csv('./data/voted-kaggled-dataset-cleaned.csv')
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           2143 non-null   int64 
 1   Title                2143 non-null   object
 2   Subtitle             2039 non-null   object
 3   Owner                2143 non-null   object
 4   Votes                2143 non-null   int64 
 5   Versions             2143 non-null   object
 6   Tags                 1603 non-null   object
 7   Data Type            2143 non-null   object
 8   Size                 2143 non-null   object
 9   License              2143 non-null   object
 10  Views                2143 non-null   object
 11  Download             2133 non-null   object
 12  Kernels              1205 non-null   object
 13  Topics               1557 non-null   object
 14  URL                  2143 non-null   object
 15  Description          2143 non-null   object
 16  Cleane

### Feature Engineering to prepare features

In [3]:
def normalize_corpus(descriptions):
    
    norm_description = []
    
    for description in descriptions:
        
        desc_tokens = [token.strip() for token in wtk.tokenize(description)]    
        #desc_tokens = [wnl.lemmatize(token) for token in desc_tokens if not token.isnumeric()]
        desc_tokens = [token for token in desc_tokens if len(token) > 1]
        desc_tokens = [token for token in desc_tokens if token not in stop_words]
        desc_tokens = list(filter(None, desc_tokens))
        
        norm_description.append(desc_tokens)
    
    return norm_description

In [4]:
processed_desc = list(raw_df['Cleaned_Description'])
norm_desc = normalize_corpus(processed_desc)
raw_df['normalized_description'] = norm_desc

In [5]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2143 entries, 0 to 2142
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              2143 non-null   int64 
 1   Title                   2143 non-null   object
 2   Subtitle                2039 non-null   object
 3   Owner                   2143 non-null   object
 4   Votes                   2143 non-null   int64 
 5   Versions                2143 non-null   object
 6   Tags                    1603 non-null   object
 7   Data Type               2143 non-null   object
 8   Size                    2143 non-null   object
 9   License                 2143 non-null   object
 10  Views                   2143 non-null   object
 11  Download                2133 non-null   object
 12  Kernels                 1205 non-null   object
 13  Topics                  1557 non-null   object
 14  URL                     2143 non-null   object
 15  Desc

In [6]:
# Feature Cleaning
#bigram = gensim.models.Phrases(raw_df['normalized_description'], min_count = 20, threshold = 20, delimiter  = '_')
#bigram_model = gensim.models.phrases.Phraser(bigram)

# Creating both unigram & bigram
#norm_corpus = []
#for doc in raw_df['normalized_description']:
#    bigram_doc = bigram_model[doc]
#    norm_corpus.append(bigram_doc)
#print(bigram_doc)

# Create a dictionary representation of the document of unigram
dictionary = gensim.corpora.Dictionary(norm_desc)

# Filter out words that occur less than 20 documents, or more than 60% of the documents.
dictionary.filter_extremes(no_below = 20, no_above = 0.6)

# Transforming corpus into bag of words vectors
bow_corpus = [dictionary.doc2bow(text) for text in norm_desc]

### fine-tuning & selecting optimal algorithm and model
* Latent Semantic Indexing: [LSI Parameters] (https://radimrehurek.com/gensim/models/lsimodel.html)
* Latent Dirichlet Allocation: [LDA parameters] (https://radimrehurek.com/gensim/models/ldamodel.html)

In [7]:
def finetuning_model(corpus, dictionary, algor, df):
    # Initialize dataframe and list to store results
    results = pd.DataFrame()
    coherence_cv_scores = []
    coherence_UMass_scores = []
    parameters = []
    algor_lst = []
    
    # Parameters
    topic_lst = [5, 6, 7, 8, 9, 10]
    random_seed = [42]
    
    # Compute hyperparameter grid
    if algor == 'LSI':
        hyperparams = {'num_topics': topic_lst, 'random_seed': random_seed, 'power_iters': [10, 50, 100]}
    elif algor == 'LDA':
        hyperparams = {'num_topics': topic_lst, 'random_state': random_seed, 'alpha': ['symmetric', 'auto'], 'passes': [1, 10], 'iterations': [50, 200, 500]}
    paramgrid = list(ParameterGrid(hyperparams))
    
    # Loop over parameter grid for LSI
    count = 0
    for params in paramgrid:
        if algor == 'LSI':
            model = LsiModel(corpus = corpus, id2word = dictionary, **params)
        elif algor == 'LDA':
            model = LdaModel(corpus = corpus, id2word = dictionary, **params)
        
        # Compute coherence score
        umass_model = CoherenceModel(model = model, corpus = corpus, dictionary = dictionary, coherence = 'u_mass')
        umass_score = umass_model.get_coherence()
        cv_model = CoherenceModel(model = model, texts = df['normalized_description'], dictionary = dictionary, coherence = 'c_v')
        cv_score = cv_model.get_coherence()
        
        # Store the results
        algor_lst.append(f'{algor}_{count}')
        parameters.append(params)
        coherence_cv_scores.append(cv_score)
        coherence_UMass_scores.append(umass_score)
        
        count += 1
        
    # Append result to result dataframe and return dataframe
    results['algorithm'] = algor_lst
    results['cv_score'] = coherence_cv_scores
    results['umass_score'] = coherence_UMass_scores
    results['parameters'] = parameters
    
    return results

In [8]:
# Perform Topic Modeling with LSI
LSI_results = finetuning_model(bow_corpus, dictionary, 'LSI', raw_df)
LSI_results

,algorithm,cv_score,umass_score,parameters
0,LSI_0,0.414374,-1.847259,"{'num_topics': 5, 'power_iters': 10, 'random_seed': 42}"
1,LSI_1,0.414374,-1.847259,"{'num_topics': 5, 'power_iters': 50, 'random_seed': 42}"
2,LSI_2,0.430570,-1.978477,"{'num_topics': 5, 'power_iters': 100, 'random_seed': 42}"
3,LSI_3,0.425324,-1.916569,"{'num_topics': 6, 'power_iters': 10, 'random_seed': 42}"
4,LSI_4,0.481299,-2.086907,"{'num_topics': 6, 'power_iters': 50, 'random_seed': 42}"
5,LSI_5,0.383058,-1.793504,"{'num_topics': 6, 'power_iters': 100, 'random_seed': 42}"
6,LSI_6,0.383131,-2.045915,"{'num_topics': 7, 'power_iters': 10, 'random_seed': 42}"
7,LSI_7,0.383131,-2.045915,"{'num_topics': 7, 'power_iters': 50, 'random_seed': 42}"
8,LSI_8,0.412386,-1.952091,"{'num_topics': 7, 'power_iters': 100, 'random_seed': 42}"
9,LSI_9,0.450456,-4.127868,"{'num_topics': 8, 'power_iters': 10, 'random_seed': 42}"


In [9]:
# Perform Topic Modeling with LDA
LDA_results = finetuning_model(bow_corpus, dictionary, 'LDA', raw_df)
LDA_results

,algorithm,cv_score,umass_score,parameters
0,LDA_0,0.393777,-1.706755,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 5, 'passes': 1, 'random_state': 42}"
1,LDA_1,0.393518,-1.895762,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 5, 'passes': 10, 'random_state': 42}"
2,LDA_2,0.386449,-1.737513,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 6, 'passes': 1, 'random_state': 42}"
3,LDA_3,0.395409,-1.911878,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 6, 'passes': 10, 'random_state': 42}"
4,LDA_4,0.373664,-1.770506,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 7, 'passes': 1, 'random_state': 42}"
5,LDA_5,0.412372,-1.866418,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 7, 'passes': 10, 'random_state': 42}"
6,LDA_6,0.382214,-1.874991,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 8, 'passes': 1, 'random_state': 42}"
7,LDA_7,0.388025,-1.984246,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 8, 'passes': 10, 'random_state': 42}"
8,LDA_8,0.376307,-1.808159,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 9, 'passes': 1, 'random_state': 42}"
9,LDA_9,0.383854,-2.218714,"{'alpha': 'symmetric', 'iterations': 50, 'num_topics': 9, 'passes': 10, 'random_state': 42}"


In [10]:
# Concat results together
full_results = pd.concat([LSI_results, LDA_results])

In [11]:
# Show results
full_results

,algorithm,cv_score,umass_score,parameters
0,LSI_0,0.414374,-1.847259,"{'num_topics': 5, 'power_iters': 10, 'random_seed': 42}"
1,LSI_1,0.414374,-1.847259,"{'num_topics': 5, 'power_iters': 50, 'random_seed': 42}"
2,LSI_2,0.430570,-1.978477,"{'num_topics': 5, 'power_iters': 100, 'random_seed': 42}"
3,LSI_3,0.425324,-1.916569,"{'num_topics': 6, 'power_iters': 10, 'random_seed': 42}"
4,LSI_4,0.481299,-2.086907,"{'num_topics': 6, 'power_iters': 50, 'random_seed': 42}"
5,LSI_5,0.383058,-1.793504,"{'num_topics': 6, 'power_iters': 100, 'random_seed': 42}"
6,LSI_6,0.383131,-2.045915,"{'num_topics': 7, 'power_iters': 10, 'random_seed': 42}"
7,LSI_7,0.383131,-2.045915,"{'num_topics': 7, 'power_iters': 50, 'random_seed': 42}"
8,LSI_8,0.412386,-1.952091,"{'num_topics': 7, 'power_iters': 100, 'random_seed': 42}"
9,LSI_9,0.450456,-4.127868,"{'num_topics': 8, 'power_iters': 10, 'random_seed': 42}"


In [12]:
# Rearrange results
results = full_results.sort_values('umass_score', ascending = True).sort_values('cv_score', ascending = False)
results

,algorithm,cv_score,umass_score,parameters
11,LSI_11,0.514781,-5.370141,"{'num_topics': 8, 'power_iters': 100, 'random_seed': 42}"
4,LSI_4,0.481299,-2.086907,"{'num_topics': 6, 'power_iters': 50, 'random_seed': 42}"
10,LSI_10,0.475780,-4.146934,"{'num_topics': 8, 'power_iters': 50, 'random_seed': 42}"
9,LSI_9,0.450456,-4.127868,"{'num_topics': 8, 'power_iters': 10, 'random_seed': 42}"
16,LSI_16,0.448502,-3.874545,"{'num_topics': 10, 'power_iters': 50, 'random_seed': 42}"
2,LSI_2,0.430570,-1.978477,"{'num_topics': 5, 'power_iters': 100, 'random_seed': 42}"
13,LSI_13,0.428188,-4.118091,"{'num_topics': 9, 'power_iters': 50, 'random_seed': 42}"
3,LSI_3,0.425324,-1.916569,"{'num_topics': 6, 'power_iters': 10, 'random_seed': 42}"
15,LSI_15,0.415627,-3.660034,"{'num_topics': 10, 'power_iters': 10, 'random_seed': 42}"
12,LSI_12,0.415557,-4.118701,"{'num_topics': 9, 'power_iters': 10, 'random_seed': 42}"


Using perplexity and coherence scores as measures to evaluate the topic model, the model would be better if
- Lower the UMass score
- Higher the Cv score

In [13]:
results.iloc[0]

algorithm                                                        LSI_11
cv_score                                                       0.514781
umass_score                                                   -5.370141
parameters     {'num_topics': 8, 'power_iters': 100, 'random_seed': 42}
Name: 11, dtype: object

In [14]:
final_df = raw_df.copy()

In [15]:
# 4c. Determine the most dominant topic for each document using the best model
best_model = LsiModel(corpus = bow_corpus, id2word = dictionary,
                      power_iters = 100, num_topics = 8,
                      random_seed = 42)

# Compute dominant topics for each document
topic_weights = []
for row in best_model[bow_corpus]:
    topic_weights.append(dict(row))
    
topic_weights_df = pd.DataFrame(topic_weights)

# List to store topic and keywords
dominant_topics = []
topic_keywords = []

for i, row in topic_weights_df.iterrows():
    sorted_topics = sorted(row.items(), key = lambda x: x[1], reverse = True)
    top_topic = f"Topic {sorted_topics[0][0]} ({sorted_topics[0][1]:.3f})"
    dominant_topics.append(top_topic)
    
    # Get top keywords for each topic
    keywords = [word for (word, prob) in best_model.show_topic(sorted_topics[0][0], topn=10)]
    topic_keywords.append(keywords)
    
# Append topic back to dataframe
final_df['Dominant_Topic'] = dominant_topics
final_df['Topic_Keywords'] = topic_keywords

In [16]:
final_df.to_csv('./data/corpus_topic_best.csv')

In [17]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 0')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].head()

,Cleaned_Description,Dominant_Topic,Topic_Keywords
1140,stick overflow dataset,Topic 0 (nan),"[university, state, college, use, player, institute, number, contain, time, file]"
1515,loan datum,Topic 0 (nan),"[university, state, college, use, player, institute, number, contain, time, file]"
1925,moist datum htpneuralnetworksandeplearningcom,Topic 0 (nan),"[university, state, college, use, player, institute, number, contain, time, file]"


In [19]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 1')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].head()

Cleaned_Description  \
1811                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [33]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 2')][['Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

Description  \
74                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [66]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 3')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

,Cleaned_Description,Dominant_Topic,Topic_Keywords
40,context vista category employmentbase nonimigrant vista temporary foreign worker united state foreign national apply isa employer must offer job submit petition isa immigration department also common isa status apply hold international student complete college high education begin work futile position follow article contain information isa process la file salary process time application process step step guide content dataset contain five year worth petition datum approximately million record overall column dataset include case status employer name worksite coordinate job title prevail wage occupation code year file information individual column refer column metadata detailed description underlie raw dataset available official datum dictionary acknowledgment office foreign labor mortification generate program datum include datum vista disclosure datum update annually available online raw datum available mess immediately suitable analysis set datum transformation perform make datum accessible quick exploration learn refer blow post complimentary r notebook inspiration number petition datum engineer job title increase time part hardware engineer job industry number datum scientist position employer file petition year,Topic 3 (1.052),"[inter, agree, interested, enjoy, much, categorical, always, music, people, use]"
731,context dataset list people involve accident city barcelona spain year till datum manage police city barcelona include several information describe content dataset compose file one contain k line every row contain several information like type injury slightly wound serious injury death include description person driver passenger pedestrian sex age location etc important dataset unload possible datum row misingnot correct description column number expedient case file number code district district code accident barcelona divide several district nom district name district code bare hood code accident every district barcelona several good nom bare name hood code carer street code every street code nom carer name street sum postal caution postal number street description seaman day week text write caravan seaman shortcode previous field also caravan description tip description type day labor festive also caravan number year I de number month nom I name month caravan de I day month description torn type round police mate morning tardy evening night horn de hour day description cause vacant text caravan describe accident case victim pedestrian say es cause del vacant desk tip vehicle implicate type vehicle accident also caravan description sexe sex victim home mean man mean woman description tip person type role accident describe victim pilot conductor passenger pasatger pedestrian vacant eat age victim description victimitzacio type injury caravan slightly wound merit let serious injury merit grow death mort cordenada tm tm coordinate cordenada tm x tm coordinate x see column could remove would loose information experience work file tell I row correct datum datum careful acknowledgment datum find open datum ban barcelona city hall open datum service owner file inspiration I unload information I believe datum share everybody research share also I always happy get feedback help,Topic 3 (1.533),"[inter, agree, interested, enjoy, much, categorical, always, music, people, use]"
2034,dataset description yet,Topic 3 (0.016),"[inter, agree, interested, enjoy, much, categorical, always, music, people, use]"
1131,context datum parliamentary agenda action take verkhovna ukrainian parliament content period th nov th oct list deputy list parliamentary fraction session day daily agenda result include total vote result individual deputy vote speech author tiding full text registration perform session day start acknowledgment source htpdataradagovuaopendatapzskl ukrainian government open datum portal thank everyone make open datum possible photo ilya cherednychenko splash inspiration

In [67]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 4')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

,Cleaned_Description,Dominant_Topic,Topic_Keywords
1858,context universal taste petrov et al create facilitate future research supervise induction syntactic structure standardize bestpractice update documentation find htpuniversaldependenciesorgupos content file depositor contain mapping trepak specific taste set universal partofspech tag universal tag verb verb tense mode noun nous common proper iron profound adj adjective adv adverse add position proposition postposition cone conjunction meet determine sum cardinal number part particle function word x foreign word type abbreviation punctuation see universal partofspech taste slav petrov dipanjan das run macdonald detail htparxivorgab zipfile contain langtagsetmap file map respective taste pus taste long universal taste eg enptbmap contain mapping english penn tree bank taste universal taste list mapping include arpadtmap bgbtbmap cacatlbmap cspdtmap dadtmap denegramap detigermap elgdtmap enbrownmap enptbmap entwetmap escastlbmap eseaglesmap esiulamap estretagermap eueuslbmap fitdtmap frparismap huszegedmap itistmap iwmilamap jakyotomap javerbmobilmap kosejongmap nlalpinomap plipipanmap ptbosquemap rurncmap slsdtmap svtalbankenmap tumetusbancimap zhctbmap zhsinicamap additional contain ready ready file universal_tagspy script use convert tag universal taste use mapping schneider entwetreadme description sweet tag mapping noah et al situation slav petrov dipanjan das run macdonald universal partofspech taste rec,Topic 4 (2.487),"[year, contain, file, element, age, child, tag, time, numerical, total]"
2043,context automatic composition computer program stand challenge since early day artificial intelligence clear solution modernday research deep learn indeed model latent context representation language prove difficult task singularly combine apply structure procedure knowledge generative fashion quickly become intractable complex domain specific language clear need research devise efficiently learn combination independent problem present basic dataset elementary mathematical function end patron program language encourage future research field benchmark deep learn content dataset contain total elementary mathematical function example input correspond output first line dataset file contain header describe content column first column label function_name follow row contain function name unique inter index next twenty column contain function_input_x function_output_x integer exclusively follow row contain stre end patron float point number execute correspond function final column label function_code contain singleline labia statement elementary mathematical function acknowledgment thank guidance student instructor machine learn berkeley research group discussion member redwood institute theoretical neuroscience inspiration hope see neutral architecture capable program new computer code without human supervision program language solve even scientific compute task license copyright c brandon trabuco permission hereby grant free charge person obtain copy software associate documentation file software deal software without restriction include without limitation right use copy modify merge publish distribute sublicense ardor sell copy software permit person software furnish subject follow condition copyright notice permission notice shall include copy substantial portion software software provide without warranty kind express imply include limited warranty merchantability fitness particular purpose noninfringement event shall author copyright holder liable claim damage liability whether action contract sort otherwise arise connection software use dealing software,Topic 4 (2.453),"[year, contain, file, element, age, child, tag, time, numerical, total]"
1105,context I scrape currently available urban dictionary page content word slang term add urban dictionary definition definition say term author user account contribute term tag list hashtag use vote downvote date date term add urban dict

In [68]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 5')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

,Cleaned_Description,Dominant_Topic,Topic_Keywords
1473,code federal regulation cf modification general permanent rule regulation sometimes call administrative law publish federal register executive department agency federal government united state subject matter title contain one individual volume update calendar year stagger basis annual update cycle follow title revise january title revise april title revise july title revise october title divide chapter usually bear name issue agency chapter far undivide part cover specific regulatory area large part may undivide support part organize section situation cf refer material section level cf publish multiple format government publish office find late version format htpwgpogovfdsysbulkdatacfr,Topic 5 (0.530),"[numerical, text, number, element, use, date, time, zero, occur, end]"
417,content report list failure commercial bank saving association saving bank since establishment record include institution name number institution charter type location headquarter city state effective date insurance fund certificate number failure transaction type total deposit total asset last report prior failure thousand dollar estimate cost resolution datum estimate loss available insure failure prior slip insure failure acknowledgment bank failure report download webster inspiration type bank institution likely fail bank failure rate change time commercial bank failure cost federal government resolve,Topic 5 (0.849),"[numerical, text, number, element, use, date, time, zero, occur, end]"
1409,world citizen treat equally one way might think measure inequality value person citizenship number country freely travel datum set contain information global reach national passport country datum identify number country passport grant isa free travel isa arrival number country passport welcome destination country datum download htpswpasportindexorgbyindividualrankphp april use follow r scrape package htpsgithubcomsdoriuspasportr information variable visarank passport index rank originally label global rank visage number country passport allow visage travel originally label individual rank vision number country passport allow visaonarival originally label individual rank visawelc number passport accept travel destination country originally label welcome rank,Topic 5 (0.701),"[numerical, text, number, element, use, date, time, zero, occur, end]"
1505,context dataset represent register unemployment city barcelona spain year till register unemployment correspond job demand pende cover last day month exclude employee want change job one readily available incompatible situation one ask specific occupation temporary agricultural beneficiary special unemployment benefit content file dataset format every row represent hood city district number hood name number citizen hood age legal age job column one per month unemployment barcelona good district every hood belong district district form several good acknowledgment datum find open datum ban barcelona city hall open datum service owner file inspiration week ago I need dataset testing purpose I unload information honest opinion datum research share everybody enjoy,Topic 5 (0.792),"[numerical, text, number, element, use, date, time, zero, occur, end]"
1395,context war one thing human specie proud recent time major political leader take step increase decrease tension ally rival part datum science community I believe field explore much affect era comfort seem delusion visualize try predict close next great war might content version dataset small initial version death toll timeline major war infer participate country much possible name war analyse death toll risefal recent year file war column name war name time time period war include start end year long war casualty number death war subsequent version good response would include weapon use participate country entry acknowledgment datum scrape wikipedia manual clean inspiration I hope dataset start butterfly effect lead u

In [73]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 6')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

,Cleaned_Description,Dominant_Topic,Topic_Keywords
420,context human many animal ability reduce suppress brain response sensory consequence result action nervous system accomplish corollary discharge forward model system reference copy impend motor plan transmit motor sensory cortex generate corollary discharge representation expect sensory consequence imminent motor act example move eye leave right brain know environment shift speak auditory cortex reduce response expect sound voice schizophrenia chronic mental illness affect people across globe one possible explanation symptom schizophrenia one problem corollary discharge process nervous system make difficult patient differentiate internally externally generate stimulus therefore study process relationship symptom illness might allow well understand abnormal brain process patient diagnosis previously publish leg experiment full report use simple button press task subject either press button immediately generate tone passively listen tone press button without generation tone study corollary discharge people schizophrenia comparison control find comparison control suppress n negative deflection leg brain wave millisecond onset sound press button generate tone compare passive playback patient schizophrenia datum set large sample application previous study specifically leg datum control patient schizophrenia combine control patient previous report method due size raw leg datum preprocese prior unload leg datum acquisition parameter experimental task identical describe paper however preprocese differ individual subject datum least follow datum process step apply order reference average ear love highway filter interposition outer channel continuous leg datum outline define paper chop continuous datum single trial epochs second task event total vaseline correction ms conical correlation analysis remove muscle highfrequency white noise artifact rejection outer single trial outline define paper removal outer component spatial independent component analysis outline define paper interposition outer channel within single trial outline define paper derive datum include eventrelate potential rep average electrode site analyze previous report include ff fez c ff ff c c picture eye calculate average across trial every sample time series separately subject electrode condition content single trial datum channel large unload subject interested type datum find one subject subject among datum file include datum preprocese step list interested compare patient schizophrenia control subject erpdatacsv file contain average rep time series subject condition electrode mention datum along subject information demographicsv could use delicate analysis prior report interested single trial categorizationprediction like graspandlift challenge face decide challenge mergedtrialdatacsv contain summary measurement nearly individual trial subject condition include acknowledgment funding study procedure initial analysis publication come national institute mental health please see grant additional detail cite nigh project number rah work relate datum study participant give write inform consent participate study receive institutional review board approval,Topic 6 (2.086),"[numerical, text, number, year, use, image, element, age, file, date]"
4,context bitcoin long run well know cryptocurency first release open source anonymous satoshi nakamoto bitcoin serve centralized medium digital exchange transaction verify record public distribute ledger blockchain without need trust record keep authority central intermediary transaction block contain cryptographic previous transaction block thus chain together serve immutable record transaction ever occur curencycomodity market bitcoin trading financial instrument soon follow public adoption bitcoin continue grow include historical bitcoin market datum min interval select bitcoin exchange trading take place happy datum mining content coincheckjpy_min_data_to_csv bitflyerjpy_m

In [75]:
final_df[final_df['Dominant_Topic'].str.contains('Topic 7')][['Cleaned_Description', 'Dominant_Topic', 'Topic_Keywords']].sample(5)

,Cleaned_Description,Dominant_Topic,Topic_Keywords
695,context referendum hold june decide whether united kingdom remain member european union leave approximately million people vote leave e referendum trout million vote cast content electoral commission publish result e referendum district region vote office national statistic provide population demographic district united kingdom census,Topic 7 (0.034),"[human, de, run, john, year, file, child, age, andrew, paul]"
43,dataset contain list video game sale great copy generate scrape vgchartzcom field include rank bank overall sale name game name platform platform game release pip etc year year game release gene gene game publisher publisher game na_sale sale north america million eu_sale sale europe million jp_sale sale japan million other_sale sale rest world million global_sale total worldwide sale script scrape datum available htpsgithubcomgregorutvgchartzscrape base beautifulsoup use patron record record drop due incomplete information,Topic 7 (1.293),"[human, de, run, john, year, file, child, age, andrew, paul]"
1381,context datum set extract fbiucr webster year population less content list variable population violent_crime_total murder_and_manslaughter forcible_rape robbery aggravated_assault property_crime_total burglar larceny_theft motor_vehicle_theft long acknowledgment I really appreciate fbiucr generosity inspiration impact population crime,Topic 7 (0.469),"[human, de, run, john, year, file, child, age, andrew, paul]"
1045,context thousand cryptocurencie spring past year predict one next etc content dataset contain daily open high low close trading volume cryptocurencie exclude bitcoin acknowledgment htpstimescaledatablobcorewindowsnetdatasetscrypto_datatargz inspiration speculative force always work cryptocurency exchange contain statistical significant feature,Topic 7 (0.359),"[human, de, run, john, year, file, child, age, andrew, paul]"
1767,datum acquire precede bent monthly sale report price average ak stick price stand manufacturer suggest retail price,Topic 7 (0.170),"[human, de, run, john, year, file, child, age, andrew, paul]"
